In [ ]:
# Use Jupyter Black for cell formatting
import jupyter_black

jupyter_black.load()

In [ ]:
import polars as pl

In [ ]:
df = pl.read_csv("input.txt", has_header=False)

df = (
    df.with_columns(
        pl.col("column_1")
        .str.split_exact("   ", 1)
        .struct.rename_fields(["first_part", "second_part"])
        .alias("fields")
    )
    .unnest("fields")
    .with_columns(
        pl.col("first_part")
        .cast(pl.Int64, strict=False)
        .alias("first_part_int")
        .sort(),
        pl.col("second_part")
        .cast(pl.Int64, strict=False)
        .alias("second_part_int")
        .sort(),
    )
    .with_columns(
        (pl.col("first_part_int") - pl.col("second_part_int")).abs().alias("part_01")
    )
)

In [ ]:
# Answer to Pat 01
df.select("part_01").sum()

In [ ]:
df_count = df["second_part_int"].value_counts()

df = (
    df.join(
        df_count,
        left_on="first_part_int",
        right_on="second_part_int",
        how="left",
    )
    .fill_null(0)
    .with_columns((pl.col("first_part_int") * pl.col("count")).alias("part_02"))
)

In [ ]:
# Answer to Pat 02
df.select("part_02").sum()